In [ ]:
"""
docker-compose down
docker-compose up --build
"""

In [ ]:

#########################################
# ROW 2 - line chart - BAR GRAPH
#########################################
# -------------------------------------------------------------------------
# Attendance Data and Line Chart
# -------------------------------------------------------------------------
# Convert 'date' column to datetime for proper grouping
attendance_data['date'] = pd.to_datetime(attendance_data['date'], errors='coerce')

# Define a function to classify attendance
def classify_attendance(row):
    if row['onleave']:
        return 'On-Leave', row['leave_type'], row['durationstart'], row['durationend']
    if pd.isna(row['timein']):
        return 'Absent', None, None, None
    try:
        timein_dt = pd.to_datetime(row['timein'], format='%H:%M:%S', errors='coerce')
        if pd.isna(timein_dt):
            return 'Unknown', None, None, None
        timein = timein_dt.time()
    except Exception as e:
        st.warning(f"Error converting timein: {e}")
        return 'Unknown', None, None, None
    cutoff = datetime.time(8, 0, 0)
    if timein >= cutoff:
        return 'Late', None, None, None
    return 'Present', None, None, None

# Apply classification function to each row
attendance_data[['Status', 'Leave Type', 'Leave Start', 'Leave End']] = attendance_data.apply(
    lambda row: pd.Series(classify_attendance(row)), axis=1
)

# Aggregate daily data: count of each status per day
status_counts = attendance_data.groupby(['date', 'Status']).size().reset_index(name='Count')
total_counts = attendance_data.groupby('date').size().reset_index(name='Total Employees')
status_counts = pd.merge(status_counts, total_counts, on='date')
status_counts['Percentage'] = (status_counts['Count'] / status_counts['Total Employees']) * 100

# Pivot table so each status becomes a separate column (excluding Absent)
status_pivot = status_counts.pivot_table(
    index='date', columns='Status', values='Percentage', aggfunc='sum'
).reset_index()

expected_statuses = ['Present', 'Late', 'On-Leave']
status_pivot = status_pivot.set_index('date').reindex(columns=expected_statuses, fill_value=0).reset_index()

# For hover details, create employee_name if missing
if 'fname' in attendance_data.columns and 'lname' in attendance_data.columns:
    attendance_data['employee_name'] = attendance_data['fname'] + ' ' + attendance_data['lname']
else:
    attendance_data['employee_name'] = ""

# Process On-Leave details for hover info
on_leave_data = attendance_data[attendance_data['Status'] == 'On-Leave']
if not on_leave_data.empty:
    on_leave_data['durationstart'] = pd.to_datetime(on_leave_data['durationstart'], errors='coerce').dt.strftime('%I:%M %p')
    on_leave_data['durationend'] = pd.to_datetime(on_leave_data['durationend'], errors='coerce').dt.strftime('%I:%M %p')
    
on_leave_summary = on_leave_data.groupby('date').agg({
    'employee_name': lambda x: ', '.join(x),
    'Leave Type': lambda x: ', '.join(x.astype(str)),
    'Leave Start': lambda x: ', '.join(x.astype(str)),
    'Leave End': lambda x: ', '.join(x.astype(str))
}).reset_index()

status_pivot = pd.merge(status_pivot, on_leave_summary, on='date', how='left')

present_summary = attendance_data[attendance_data['Status'] == 'Present'].groupby('date').agg(
    present_names=('employee_name', lambda x: ', '.join(x)),
    present_count=('employee_name', 'count')
).reset_index()

late_summary = attendance_data[attendance_data['Status'] == 'Late'].groupby('date').agg(
    late_names=('employee_name', lambda x: ', '.join(x)),
    late_count=('employee_name', 'count')
).reset_index()

status_pivot = pd.merge(status_pivot, present_summary, on='date', how='left')
status_pivot = pd.merge(status_pivot, late_summary, on='date', how='left')

fig_line = px.line(
    status_pivot, 
    x='date', 
    y=expected_statuses, 
    title='Daily Employee Attendance Rates',
    labels={'value': 'Percentage', 'date': 'Date'},
    markers=True
)
fig_line.update_layout(
    xaxis=dict(tickformat="%B %d, %Y")
)
for trace in fig_line.data:
    if trace.name == 'Present':
        trace.hovertemplate = (
            'Date: %{x|%B %d, %Y}<br>'
            'Present: %{y:.2f}%<br>'
            'Employee Names: %{customdata[0]}<br>'
            'Count: %{customdata[1]}<br>'
            '<extra></extra>'
        )
        customdata = status_pivot[['present_names', 'present_count']].fillna('').values
        trace.customdata = customdata
    elif trace.name == 'Late':
        trace.hovertemplate = (
            'Date: %{x|%B %d, %Y}<br>'
            'Late: %{y:.2f}%<br>'
            'Employee Names: %{customdata[0]}<br>'
            'Count: %{customdata[1]}<br>'
            '<extra></extra>'
        )
        customdata = status_pivot[['late_names', 'late_count']].fillna('').values
        trace.customdata = customdata

# -------------------------------------------------------------------------
# Task Distribution by State - Bar Chart
# -------------------------------------------------------------------------
def load_data():
    query = "SELECT state, COUNT(*) as count FROM Story_points GROUP BY state ORDER BY count DESC"
    return pd.read_sql(query, engine)

data = load_data()
fig_bar = px.bar(data, x='state', 
                 y='count', 
                 labels={'state': 'Task State', 'count': 'Number of Tasks'}, 
                 height=400,
                 width=50,
                 title="Task Distribution by State"
                 )

# -------------------------------------------------------------------------
# Place the two visualizations in two columns
# -------------------------------------------------------------------------
col1, col2 = st.columns(2)

with col1:
    st.plotly_chart(fig_line, use_container_width=True)

with col2:
    st.plotly_chart(fig_bar, use_container_width=True)
